# Introduction

NASA's [AVIRIS-NG](https://avirisng.jpl.nasa.gov) comes as ENVI image format, which might not be amenable to some software and the cloud. This tutorial demonstrates simple conversions to various formats such as [GeoTIFFs](https://www.ogc.org/standards/geotiff), [CoGs](https://www.cogeo.org/), [netCDF](https://www.unidata.ucar.edu/software/netcdf/conventions.html), and [zarr](https://zarr.readthedocs.io/en/stable/).

## Sample file
We will use one Level 2 reflectance file `ang20170706t180635_corr_v2p9_img`, which can be downloaded from [this portal](https://avirisng.jpl.nasa.gov/dataportal/). The `corr` in the file name means that this is a orthocorrected, scaled reflectance image file. The `v2p9` is a version marker. For more information about the file name, read the [filenaming convention](https://avirisng.jpl.nasa.gov/dataportal/ANG_L1B_L2_Data_Product_Readme_v02.txt).

## Converting to netCDF

In [ ]:
import rioxarray
avirisng_f = 'ang20170706t180635_corr_v2p9_img'
avirisng = rioxarray.open_rasterio(avirisng_f, driver='ENVI', parse_coordinates=True)
avirisng.attrs['long_name'] = 'orthocorrected scaled reflectance'

Now, the `avirisng` variables contains all bands and attribute information as xarray. 

We will add a longitude and latitude coordinates as well.

In [ ]:
from pyproj import Transformer
import xarray as xr

transformer = Transformer.from_crs(avirisng.rio.crs, "epsg:4326", always_xy=True) 
lon1, lat1 = transformer.transform(avirisng.xc, avirisng.yc)
avirisng = avirisng.assign_coords(lon=xr.DataArray(lon1, dims=avirisng.xc.dims))
avirisng = avirisng.assign_coords(lat=xr.DataArray(lat1, dims=avirisng.yc.dims))

We will first give a name ("corr") to the dataarray and then convert to a netCDF4 file.

In [ ]:
avirsng_ds=avirisng.to_dataset(name='corr')

In [ ]:
avirsng_ds.to_netcdf(avirisng_f + '.nc')